# General Usage

In [2]:
%load_ext autoreload
%autoreload 2
%pprint off

Pretty printing has been turned OFF


In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
%pprint

Pretty printing has been turned ON


### Preamble

[!] use only if you have **NOT** installed FHIR PACK via `pip`, `pipenv` or similar in your environment but you have instead cloned the project and are now in the `/examples` directory

In [5]:
import sys,os
sys.path.append(os.getcwd()+'/../src/')
# sys.path

### Imports

In [6]:
import fhirpack			as fp
import pandas			as pd
import fhirpy
import numpy			as np
import fhir.resources	as fr
import json
from tqdm import tqdm
from fhirpack.constants import CONFIG

### Useful Paths for Data Storage 

In [7]:
DATAPATH=CONFIG.get('DATAPATH')
RESPATH=f"{DATAPATH}/fhir/"

### FHIR Server Connection

### CLI Usage

In [130]:
!python -m fhirpack.cli -e .env.example "-o" "getPatients" -p "given = Chalmers" #-v

SEARCH[Patient]> : 100%|██████████████████████████| 1/1 [00:01<00:00,  1.38s/it]
                                                data Patient.id
0  {'resourceType': 'Patient', 'id': '1567099', '...    1567099


In [131]:
# public FHIR test servers may be unavailable sometimes
# if you prefer to test FHIRPACK using a locally running FHIR server
# you may use our provided Docker environment by changing into the
# cloned repository and running `docker-compose up``

!fp -e .env.docker -o "getPatients" -p "given = Max" #-v

SEARCH[Patient]> : 100%|██████████████████████████| 3/3 [00:01<00:00,  2.13it/s]
                                                data Patient.id
0  {'resourceType': 'Patient', 'id': '146053', 'm...     146053
1  {'resourceType': 'Patient', 'id': '146255', 'm...     146255
2  {'resourceType': 'Patient', 'id': '146496', 'm...     146496


### From Custom Environment File

In [132]:
pack = fp.PACK(envFile='.env.docker')

for i,e in pack.getPatients(['146053']).data.items():
	print(
		json.dumps(e.serialize(), indent=4, sort_keys=True)[:200],
		'\n...'
	)

GET[Patient]> : 100%|██████████| 1/1 [00:00<00:00, 55.44it/s]

{
    "address": [
        {
            "city": "Walpole",
            "country": "US",
            "extension": [
                {
                    "extension": [
                        {
      
...


In [133]:
!cat .env.example


SCHEMA=http
PORT=80

#DOMAIN=${SCHEMA}://hapi.fhir.org
#server has moved temporarily https://github.com/hapifhir/hapi-fhir/issues/1171#issuecomment-1179728061

DOMAIN=${SCHEMA}://test.hapifhir.io

#DOMAIN=${SCHEMA}://example.com:${PORT}
# only use port if the server is running on a non-standard port 
# and this is reflected in the FHIR pagination links
# see https://gitlab.com/fhirpack/main/-/issues/75 for more information

APIBASE=${DOMAIN}/baseR4

#AUTH_METHOD=
#AUTH_PARAMS_PRESET=
#LOGINURL=${DOMAIN}/oauth/token
#OAUTH_TOKEN_ENDPOINT=${DOMAIN}/oauth/token
#OAUTH_USERNAME=username
#OAUTH_PASSWORD=password
#OAUTH_TOKEN=token

LOGSPATH=./logs
DATAPATH=./data

#EXTRACTION_BASE_TOKEN_DICOM=${OAUTH_TOKEN}



### From Default .env Environment File

In [134]:
# pack = fp.PACK()
# pack.client.authorization

### Manual Setup

In [8]:

# pack  = fp.PACK("http://hapi.fhir.org/baseR4")
# pack  = fp.PACK(envFile=".env.example")
pack  = fp.PACK("http://test.hapifhir.io/baseR4")
pack.getPatients(['2918556'])

# pack  = fp.PACK("http://127.0.0.1:42112/hapi-fhir-jpaserver/fhir")
# pack.getPatients(['146053'])

GET[Patient]> : 100%|██████████| 1/1 [00:00<00:00,  3.09it/s]


,data,Patient.id
0,"{'resourceType': 'Patient', 'id': '2918556', '...",2918556


In [136]:
#testing with another less reliable FHIR server
# pack  = fp.PACK("http://test.fhir.org/r4/")
# pack.getPatients(['11']).data.iloc[0]

### Testing with A Docker FHIR Server and Synthetic Data

In [137]:
# pack  = fp.PACK(envFile=".env.docker")
# pack  = fp.PACK(
# 			"http://127.0.0.1:42112/hapi-fhir-jpaserver/fhir",
# 			ignoreEnvFile=True
# 		)

# pack.getPatients(['146053'])

GET[Patient]> : 100%|██████████| 1/1 [00:00<00:00, 61.56it/s]


,data,Patient.id
0,"{'resourceType': 'Patient', 'id': '146053', 'm...",146053


In [138]:
# alternatively, if you're using our Dockerfile to run your FHIR server
# remember to run "docker-compose up" in your local clone of FHIRPACK first  

# pack  = fp.PACK("http://127.0.0.1:42112/hapi-fhir-jpaserver/fhir")
# pack.getPatients(['1']).data.iloc[0].serialize()
# pack.getPatients(['146053'])


GET[Patient]> : 100%|██████████| 1/1 [00:00<00:00, 52.43it/s]


{'resourceType': 'Patient',
 'id': '1',
 'meta': {'versionId': '1',
  'lastUpdated': '2020-06-11T15:26:55.251+00:00',
  'source': '#JlcA0bg5xBW1iwXq',
  'tag': [{'system': 'https://smarthealthit.org/tags',
    'code': 'synthea-5-2019'}]},
 'text': {'status': 'generated',
  'div': '<div xmlns="http://www.w3.org/1999/xhtml">Generated by <a href="https://github.com/synthetichealth/synthea">Synthea</a>.Version identifier: v2.4.0-100-g26a4b936\n .   Person seed: 8766734163625651911  Population seed: 1559319163074</div>'},
 'extension': [{'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-race',
   'extension': [{'url': 'ombCategory',
     'valueCoding': {'system': 'urn:oid:2.16.840.1.113883.6.238',
      'code': '2106-3',
      'display': 'White'}},
    {'url': 'text', 'valueString': 'White'}]},
  {'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity',
   'extension': [{'url': 'ombCategory',
     'valueCoding': {'system': 'urn:oid:2.16.840.1.113883.6.238',


### Introductory Example

In [193]:
# for patients whose lastname is Koepp, find the root identity,
# get their associated diagnostic reports and report their ID and status

pack.getPatients(searchParams={"family":"koepp"})\
	.getDiagnosticReports()\
		.getPatients()\
			.getRootPatients()\
				.gatherSimplePaths(['id','birthDate'])

SEARCH[Patient]> : 100%|██████████| 7/7 [00:00<00:00, 21.74it/s]


,id,birthDate
0,7ca61bcb-80a6-43fb-ad49-7334df273d39,2007-11-18
1,7ca61bcb-80a6-43fb-ad49-7334df273d39,2007-11-18
2,7ca61bcb-80a6-43fb-ad49-7334df273d39,2007-11-18
3,7ca61bcb-80a6-43fb-ad49-7334df273d39,2007-11-18
4,7ca61bcb-80a6-43fb-ad49-7334df273d39,2007-11-18
...,...,...
64,2744581,1956-08-03
65,2744581,1956-08-03
66,2744581,1956-08-03
67,2744581,1956-08-03


### FHIR PACK Usage 

### References

In [196]:
patientReference = pack.getReferences(['Patient/647487'])
patientReference.data.iloc[0].to_resource().serialize()

{'resourceType': 'Patient',
 'id': '647487',
 'meta': {'versionId': '1',
  'lastUpdated': '2020-03-20T21:38:58.974+00:00',
  'source': '#bE19PhJ3nQMhB1xy'},
 'text': {'status': 'generated',
  'div': '<div xmlns="http://www.w3.org/1999/xhtml"><div class="hapiHeaderText">Series_Status: Not complete Evaluation_Status_1.0: Valid Evaluation_Status_2.0: Valid Evaluation_Status_3.0: Valid Evaluation_Status_4.0: Valid Evaluation_Status_5.0: Valid <b># 5 PENTACEL AT ≥ 4 YRS </b></div><table class="hapiPropertyTable"><tbody><tr><td>Date of birth</td><td><span>20 January 2016</span></td></tr></tbody></table></div>'},
 'name': [{'family': '# 5 Pentacel at ≥ 4 yrs',
   'given': ['Series_Status: Not complete',
    'Evaluation_Status_1.0: Valid',
    'Evaluation_Status_2.0: Valid',
    'Evaluation_Status_3.0: Valid',
    'Evaluation_Status_4.0: Valid',
    'Evaluation_Status_5.0: Valid']}],
 'gender': 'female',
 'birthDate': '2016-01-20'}

In [197]:
# getResources applied to References simplifies the conversion
# of FHIR/FHIRpy References to Resources

patientReference.getResources()

GET[None]> : 100%|██████████| 1/1 [00:00<00:00,  3.57it/s]


,data,Patient.id
0,"{'resourceType': 'Patient', 'id': '647487', 'm...",647487


### Resources

### Reference to Resource

In [199]:
multiPatientReference = pack.getReferences(
    [
        'Patient/1849165',
        'Patient/647487'
    ]
)

multiPatientReference.data.apply( lambda x:x.id )

0    1849165
1     647487
Name: data, dtype: object

In [200]:
multiPatientReference.getResources()

GET[None]> : 100%|██████████| 2/2 [00:00<00:00,  3.57it/s]


,data,Patient.id
0,"{'resourceType': 'Patient', 'id': '1849165', '...",1849165
1,"{'resourceType': 'Patient', 'id': '647487', 'm...",647487


### Direct Resource

In [201]:
pack.getPatients(
	searchParams={"given":"max"}
)

SEARCH[Patient]> :  21%|██        | 100/476 [00:04<00:16, 22.77it/s]


,data,Patient.id
0,"{'resourceType': 'Patient', 'id': '2583214', '...",2583214
1,"{'resourceType': 'Patient', 'id': '2583215', '...",2583215
2,"{'resourceType': 'Patient', 'id': '1709020', '...",1709020
3,"{'resourceType': 'Patient', 'id': '1638431', '...",1638431
4,"{'resourceType': 'Patient', 'id': '1631503', '...",1631503
...,...,...
95,"{'resourceType': 'Patient', 'id': '2651939', '...",2651939
96,"{'resourceType': 'Patient', 'id': '2651964', '...",2651964
97,"{'resourceType': 'Patient', 'id': '2651987', '...",2651987
98,"{'resourceType': 'Patient', 'id': '2651998', '...",2651998


In [207]:
pack.getPatients(
	searchParams={"family":"Betterhalf"}
).shape

SEARCH[Patient]> : 100%|██████████| 12/12 [00:01<00:00,  9.67it/s]


(12, 2)

In [208]:
multiPatientResource = pack.getResources(
    ['Patient/1849165',
     'Patient/647487']
)

multiPatientResource

GET[None]> : 100%|██████████| 2/2 [00:00<00:00,  3.59it/s]


,data,Patient.id
0,"{'resourceType': 'Patient', 'id': '1849165', '...",1849165
1,"{'resourceType': 'Patient', 'id': '647487', 'm...",647487


### Alterntives to Serialization and Element Access

In [209]:
patientResource = patientReference.data.iloc[0].to_resource()

patientResource['name'][0]['given']

patientResource.serialize().get('birthDate')

['Series_Status: Not complete',
 'Evaluation_Status_1.0: Valid',
 'Evaluation_Status_2.0: Valid',
 'Evaluation_Status_3.0: Valid',
 'Evaluation_Status_4.0: Valid',
 'Evaluation_Status_5.0: Valid']

'2016-01-20'

In [210]:
multiPatientReference.summary
multiPatientReference.getResources().summary

,id,resourceType
0,1849165,Patient
1,647487,Patient


GET[None]> : 100%|██████████| 2/2 [00:00<00:00,  3.47it/s]


,id,name.given,name.family,birthDate,city,state,country
0,1849165,"[[Peter, James], [Jim], [Peter, James]]","[Chalmers, Windsor]",1974-12-25,None,None,None
1,647487,"[[Series_Status: Not complete, Evaluation_Stat...",[# 5 Pentacel at ≥ 4 yrs],2016-01-20,None,None,None


In [211]:
multiPatientResource[:1].pretty

[
    {
        "_birthDate": {
            "extension": [
                {
                    "url": "http://hl7.org/fhir/StructureDefinition/patient-birthTime",
                    "valueDateTime": "1974-12-25T14:35:45-05:00"
                }
            ]
        },
        "active": true,
        "address": [
            {
                "city": "PleasantVille",
                "district": "Rainbow",
                "line": [
                    "534 Erewhon St"
                ],
                "period": {
                    "start": "1974-12-25"
                },
                "postalCode": "3999",
                "state": "Vic",
                "text": "534 Erewhon St PeasantVille, Rainbow, Vic  3999",
                "type": "both",
                "use": "home"
            }
        ],
        "birthDate": "1974-12-25",
        "contact": [
            {
                "address": {
                    "city": "PleasantVille",
                    "district": "Rainbow"

In [214]:
pack.getDiagnosticReports(
    ['Patient/647487']
)[:1].keys


GET[DiagnosticReport]> : 100%|██████████| 1/1 [00:00<00:00,  3.36it/s]

resourceType
id
meta
meta.versionId
meta.lastUpdated
meta.source
text
text.status
text.div
name
name.family
name.given
gender
birthDate


### DataFrame Operations

In [215]:
multiPatientReference[:1]

multiPatientReference[-1:]

multiPatientReference.values

multiPatientReference.info()

,data
0,{'reference': 'Patient/1849165'}


,data
1,{'reference': 'Patient/647487'}


array([[<SyncFHIRReference Patient/1849165>],
       [<SyncFHIRReference Patient/647487>]], dtype=object)

<class 'fhirpack.base.Frame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   data    2 non-null      object
dtypes: object(1)
memory usage: 144.0+ bytes


### pack.base.Frame

In [216]:
fp.base.Frame( [[patientResource]], columns=['data'])

,data
0,"{'resourceType': 'Patient', 'id': '647487', 'm..."


### Extraction

### Patient Extraction

### fhirpack.extraction.getPatients

In [218]:
pack.getPatients(
	[
		'647487'
	]
)


GET[Patient]> : 100%|██████████| 1/1 [00:00<00:00,  2.97it/s]


,data,Patient.id
0,"{'resourceType': 'Patient', 'id': '647487', 'm...",647487


In [219]:
pack.getPatients(
	[
		'647487'
	]
).data.item().serialize()

GET[Patient]> : 100%|██████████| 1/1 [00:00<00:00,  3.05it/s]


{'resourceType': 'Patient',
 'id': '647487',
 'meta': {'versionId': '1',
  'lastUpdated': '2020-03-20T21:38:58.974+00:00',
  'source': '#bE19PhJ3nQMhB1xy'},
 'text': {'status': 'generated',
  'div': '<div xmlns="http://www.w3.org/1999/xhtml"><div class="hapiHeaderText">Series_Status: Not complete Evaluation_Status_1.0: Valid Evaluation_Status_2.0: Valid Evaluation_Status_3.0: Valid Evaluation_Status_4.0: Valid Evaluation_Status_5.0: Valid <b># 5 PENTACEL AT ≥ 4 YRS </b></div><table class="hapiPropertyTable"><tbody><tr><td>Date of birth</td><td><span>20 January 2016</span></td></tr></tbody></table></div>'},
 'name': [{'family': '# 5 Pentacel at ≥ 4 yrs',
   'given': ['Series_Status: Not complete',
    'Evaluation_Status_1.0: Valid',
    'Evaluation_Status_2.0: Valid',
    'Evaluation_Status_3.0: Valid',
    'Evaluation_Status_4.0: Valid',
    'Evaluation_Status_5.0: Valid']}],
 'gender': 'female',
 'birthDate': '2016-01-20'}

In [220]:
for i, r in pack.getPatients(['647487','1849165']).iterrows():
    print (f"Row {i} with Patient ID {r.data.id}")

GET[Patient]> : 100%|██████████| 2/2 [00:00<00:00,  3.80it/s]

Row 0 with Patient ID 647487
Row 1 with Patient ID 1849165


In [221]:
getPatientsAsList=pack.getPatients(['647487','1849165']).cast('list')

getPatientsAsList

GET[Patient]> : 100%|██████████| 2/2 [00:00<00:00,  3.85it/s]


[[<SyncFHIRResource Patient/647487>, '647487'],
 [<SyncFHIRResource Patient/1849165>, '1849165']]

In [222]:
getPatientsAsList

[[<SyncFHIRResource Patient/647487>, '647487'],
 [<SyncFHIRResource Patient/1849165>, '1849165']]

In [223]:
[e[1] for e in getPatientsAsList]

[e[0].id for e in getPatientsAsList]

['647487', '1849165']

['647487', '1849165']

### Patients as Operand and Extracting other Resources 

In [224]:
patients = pack.getPatients(
    ['9ac622c4-61c8-40b4-b6d9-06cfeb3d6995',
    '1849165',
    '649231',
    '650224']
)


GET[Patient]> : 100%|██████████| 4/4 [00:01<00:00,  3.34it/s]


### fhirpack.extraction.getRootPatient

In [225]:
patients.getRootPatients()

,data,RootPatient.id,Patient.id
0,"{'resourceType': 'Patient', 'id': '9ac622c4-61...",9ac622c4-61c8-40b4-b6d9-06cfeb3d6995,9ac622c4-61c8-40b4-b6d9-06cfeb3d6995
1,"{'resourceType': 'Patient', 'id': '1849165', '...",1849165,1849165
2,"{'resourceType': 'Patient', 'id': '649231', 'm...",649231,649231
3,"{'resourceType': 'Patient', 'id': '650224', 'm...",650224,650224


In [16]:
pack.getRootPatients(
    [
        'b558da74-7756-4845-87d5-d1cca8b79a62',
    ],
).getLinkedPatients()

# .gatherSimplePaths(["id"]).id.unique()

SEARCH[Patient]> : 100%|██████████| 1/1 [00:00<00:00,  3.89it/s]


,data,LinkedPatient.id,RootPatient.id,Patient.id
0,"{'resourceType': 'Patient', 'id': 'b558da74-77...",b558da74-7756-4845-87d5-d1cca8b79a62,9ac622c4-61c8-40b4-b6d9-06cfeb3d6995,b558da74-7756-4845-87d5-d1cca8b79a62


In [227]:
pack.getResources(
	[
		'Patient/b558da74-7756-4845-87d5-d1cca8b79a62'
	]
).getRootPatients()

SEARCH[RootPatient]> : 100%|██████████| 1/1 [00:00<00:00,  4.04it/s]


,data,RootPatient.id,Patient.id
0,"{'resourceType': 'Patient', 'id': '9ac622c4-61...",9ac622c4-61c8-40b4-b6d9-06cfeb3d6995,b558da74-7756-4845-87d5-d1cca8b79a62


In [228]:
rootPatients = patients.getRootPatients()

rootPatients.gatherSimplePaths(['id','name.given','name.family','birthDate'])

,id,name.given,name.family,birthDate
0,9ac622c4-61c8-40b4-b6d9-06cfeb3d6995,[[TestGiven]],[TestFamily],1981-01-01
1,1849165,"[[Peter, James], [Jim], [Peter, James]]","[Chalmers, Windsor]",1974-12-25
2,649231,"[[Series_Status: Complete, Evaluation_Status_1...",[Dose # 1 PCV 13 at age 12 mos # 2 at 1 yr 8 w...,2018-12-21
3,650224,"[[Series_Status: Not complete, Evaluation_Stat...",[# 1 at age ≥ 13 years old],2007-02-08


In [229]:
rootPatients.data[0].name

[{'family': 'TestFamily', 'given': ['TestGiven']}]

In [230]:
for link in rootPatients.data[0]['link']:
	lpat=link.other.to_resource()
	print(lpat.id)

b558da74-7756-4845-87d5-d1cca8b79a62


### fhirpack.extraction.getLinkedPatients

In [9]:
pack.getLinkedPatients(
	[
		'9ac622c4-61c8-40b4-b6d9-06cfeb3d6995',
	]
)

SEARCH[Patient]> : 100%|██████████| 1/1 [00:00<00:00,  3.65it/s]


,data,LinkedPatient.id,RootPatient.id,Patient.id
0,"{'resourceType': 'Patient', 'id': 'b558da74-77...",b558da74-7756-4845-87d5-d1cca8b79a62,9ac622c4-61c8-40b4-b6d9-06cfeb3d6995,9ac622c4-61c8-40b4-b6d9-06cfeb3d6995


### fhirpack.extraction.getConditions

### Conditions

In [232]:
conditions = pack.getConditions(
	searchParams={
		'code':'44465007',
		'_sort': '-onset-date',
	}
)[:5]

conditions

SEARCH[Condition]> : 100%|██████████| 36/36 [00:00<00:00, 67.90it/s]


,data,Condition.id
0,"{'resourceType': 'Condition', 'id': '1584692',...",1584692
1,"{'resourceType': 'Condition', 'id': '1591180',...",1591180
2,"{'resourceType': 'Condition', 'id': '1402386',...",1402386
3,"{'resourceType': 'Condition', 'id': '1403562',...",1403562
4,"{'resourceType': 'Condition', 'id': '1495528',...",1495528


### Patients for Conditions

In [233]:
conditions.getPatients().gatherSimplePaths(['id','name.given','name.family'])


SEARCH[Patient]> : 100%|██████████| 5/5 [00:00<00:00, 17.46it/s]


,id,name.given,name.family
0,1402076,[[Phyliss]],[Nyambura]
1,1403421,[[Stefania]],[Bethwell]
2,1495412,"[[Hallie315], [Hallie315]]","[Donnelly343, Osinski784]"
3,1583909,[[Shayne60]],[Miller503]
4,1590885,[[Ty725]],[Sporer811]


In [234]:
conditions\
	.getPatients()\
			.data.apply(lambda x:x.id)\
				.to_csv(f"data.ignore")

SEARCH[Patient]> : 100%|██████████| 5/5 [00:00<00:00, 18.43it/s]


In [235]:
!cat data.ignore

,data
0,1402076
1,1403421
2,1495412
3,1583909
4,1590885


In [236]:
conditions = pd.read_csv( f"data.ignore", index_col=0)
conditions.shape
conditions

(5, 1)

,data
0,1402076
1,1403421
2,1495412
3,1583909
4,1590885


### Conditions for Patient

In [237]:
pack.getPatients(['8cbf1128-3644-47a1-9cc8-05f1aac6071d']).getConditions()

SEARCH[Condition]> : 100%|██████████| 5/5 [00:00<00:00, 18.07it/s]


,data,Condition.id,Patient.id
0,"{'resourceType': 'Condition', 'id': '2b7a4d69-...",2b7a4d69-00fd-4574-86da-bdf46b21b72b,{'reference': 'Patient/8cbf1128-3644-47a1-9cc8...
1,"{'resourceType': 'Condition', 'id': '0fc73e7c-...",0fc73e7c-18a8-47d8-bdc9-82df7f06e4a8,{'reference': 'Patient/8cbf1128-3644-47a1-9cc8...
2,"{'resourceType': 'Condition', 'id': 'cb1bab6c-...",cb1bab6c-581b-47f4-8036-429462224f55,{'reference': 'Patient/8cbf1128-3644-47a1-9cc8...
3,"{'resourceType': 'Condition', 'id': 'bc301bbf-...",bc301bbf-d354-4422-abe8-f7fe9c48d83c,{'reference': 'Patient/8cbf1128-3644-47a1-9cc8...
4,"{'resourceType': 'Condition', 'id': '5db4866a-...",5db4866a-c48a-4b62-923b-96daa243226b,{'reference': 'Patient/8cbf1128-3644-47a1-9cc8...


### fhipack.extraction.getEpisodesOfCare

In [238]:
rootPatients[:2]

,data,RootPatient.id,Patient.id
0,"{'resourceType': 'Patient', 'id': '9ac622c4-61...",9ac622c4-61c8-40b4-b6d9-06cfeb3d6995,9ac622c4-61c8-40b4-b6d9-06cfeb3d6995
1,"{'resourceType': 'Patient', 'id': '1849165', '...",1849165,1849165


In [239]:
pack.getPatients(['P0522-patientBSJ1']).getEpisodesOfCare()

SEARCH[EpisodeOfCare]> : 100%|██████████| 1/1 [00:00<00:00,  3.17it/s]


,data,EpisodeOfCare.id,Patient.id
0,"{'resourceType': 'EpisodeOfCare', 'id': 'P0522...",P0522-eLTSS-episode-Initial,None


### fhipack.extraction.getFamilyMemberHistories

In [240]:
pack.getPatients(['Patient/2866670']).getFamilyMemberHistories()

SEARCH[FamilyMemberHistory]> : 100%|██████████| 1/1 [00:00<00:00,  3.46it/s]


,data,FamilyMemberHistory.id,Patient.id


### fhipack.extraction.getMedicationAdministrations

In [241]:
pack.getPatients(['Patient/31678']).getMedicationAdministrations()[:5]

SEARCH[MedicationAdministration]> : 100%|██████████| 1/1 [00:00<00:00,  4.04it/s]


,data,MedicationAdministration.id,Patient.id
0,"{'resourceType': 'MedicationAdministration', '...",31675,{'reference': 'Patient/31678'}


### fhipack.extraction.getMedicationRequests

In [242]:
pack.getPatients(['Patient/202205uscore-patient-example-1']).getMedicationRequests()

SEARCH[MedicationRequest]> : 100%|██████████| 3/3 [00:00<00:00, 10.68it/s]


,data,MedicationRequest.id,Patient.id


### fhipack.extraction.getObservations

In [243]:
pack\
	.getPatients(['Patient/258974'])\
		.getConditions(searchParams={"code":"44465007"})\
				.getPatients(searchParams={"family":"Keebler762"})

SEARCH[Patient]> : 100%|██████████| 1/1 [00:00<00:00,  3.89it/s]


,data,Patient.id,Condition.id
0,"{'resourceType': 'Patient', 'id': '258974', 'm...",258974,259094


In [244]:

from datetime import datetime

interesting = pack.getPatients(['Patient/2164033']).\
    getObservations(
        searchParams={
            "code":"http ://loinc.org|29463-7",
            }
    )\
        .gatherSimplePaths(['id','issued','code.coding.code'])\
            # .explode(['id','issued','code.coding.code'])

interesting.issued = pd.to_datetime(interesting.issued).dropna().apply(lambda x:x.replace(tzinfo=None))
interesting[interesting.issued> datetime.fromisoformat('2021-01-01T00:00:00')]

SEARCH[Observation]> : 100%|██████████| 1/1 [00:00<00:00,  4.00it/s]


,id,issued,code.coding.code
0,2164039,2021-06-05 05:30:23.291,[29463-7]


### fhirpack.extraction.getDiagnosticReports

In [245]:
diagnosticReports=pack\
	.getPatients(["Patient/9b8c1901-62ee-48a7-8229-b771d59f1e5f"])\
		.getDiagnosticReports(
			searchParams={}
			)\
			# .explode()

diagnosticReports

SEARCH[DiagnosticReport]> : 100%|██████████| 12/12 [00:00<00:00, 25.38it/s]


,data,DiagnosticReport.id,Patient.id
0,"{'resourceType': 'DiagnosticReport', 'id': '12...",122da94f-d24b-47ef-8409-58eb023cfb78,{'reference': 'Patient/9b8c1901-62ee-48a7-8229...
1,"{'resourceType': 'DiagnosticReport', 'id': '5a...",5a1a396b-783c-495c-ace9-cedce2f0c335,{'reference': 'Patient/9b8c1901-62ee-48a7-8229...
2,"{'resourceType': 'DiagnosticReport', 'id': '2d...",2d363733-58cb-4aba-b73f-487181359418,{'reference': 'Patient/9b8c1901-62ee-48a7-8229...
3,"{'resourceType': 'DiagnosticReport', 'id': '01...",013d71cd-e38e-497c-9382-b21fb5eb5f53,{'reference': 'Patient/9b8c1901-62ee-48a7-8229...
4,"{'resourceType': 'DiagnosticReport', 'id': '47...",471713d6-69eb-4fcf-900d-2eccf9d10f69,{'reference': 'Patient/9b8c1901-62ee-48a7-8229...
5,"{'resourceType': 'DiagnosticReport', 'id': 'b1...",b17c7848-f6c5-49a7-bd0a-f018ecc422de,{'reference': 'Patient/9b8c1901-62ee-48a7-8229...
6,"{'resourceType': 'DiagnosticReport', 'id': '58...",58c69ea7-b4be-40a5-a59e-459b8fed660a,{'reference': 'Patient/9b8c1901-62ee-48a7-8229...
7,"{'resourceType': 'DiagnosticReport', 'id': '87...",873433a5-68ed-42da-a6a0-d0ef3f7fd9ef,{'reference': 'Patient/9b8c1901-62ee-48a7-8229...
8,"{'resourceType': 'DiagnosticReport', 'id': '8c...",8c5adc06-2c8c-431f-9492-d7c9ef99635d,{'reference': 'Patient/9b8c1901-62ee-48a7-8229...
9,"{'resourceType': 'DiagnosticReport', 'id': 'ac...",ac83cf63-9ea8-43ce-89a8-21a6b3d3319d,{'reference': 'Patient/9b8c1901-62ee-48a7-8229...


In [246]:
paths=[
	"subject",
	"presentedForm.contentType",
	"presentedForm.data",
	"presentedForm.url",
	"presentedForm.title",
	"presentedForm.creation"
]



diagnosticReports=pack.getDiagnosticReports(
	searchParams={
		"_id":"19bdf90a-8ca4-4921-8aeb-2bf3423aaf09",
		# "identifier":"|",
		# "_content":"covid19",
		# "code":"|",
		# "issued__gt":"2010-01-01",
		# "issued__lt":"2011-01-01"
	}
)

diagnosticReports.gatherSimplePaths(paths)

SEARCH[DiagnosticReport]> : 100%|██████████| 1/1 [00:00<00:00,  3.72it/s]


,subject,presentedForm.contentType,presentedForm.data,presentedForm.url,presentedForm.title,presentedForm.creation
0,{'reference': 'Patient/e92abcdc-a300-41e0-aa0f...,[text/plain],[Q2pJd01UQXRNVEV0TVRNS0NpTWdRMmhwWldZZ1EyOXRjR...,None,None,None


### fhirpack.extraction.getURLBytes

In [247]:
paths=[
    'id',
    'subject.reference',
    'presentedForm.url'
]

diagnosticReports = pack.getPatients(
    ['Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc']
    )\
        .getDiagnosticReports()[:10]

diagnosticReports=diagnosticReports.gatherSimplePaths(paths)
diagnosticReports=diagnosticReports.explode('presentedForm.url')
diagnosticReports

# diagnosticReports['path']=diagnosticReports['id']+'_'+diagnosticReports['presentedForm.url'].str.split('/').str[-1:].str[0]
# diagnosticReports['data']=diagnosticReports.getURLBytes(operateOnCol='presentedForm.url')
# diagnosticReports.sendBytesToFile()

SEARCH[DiagnosticReport]> : 100%|██████████| 11/11 [00:00<00:00, 21.03it/s]


,id,subject.reference,presentedForm.url
0,1ca4c411-116d-49d7-82e0-cde6bff90cfd,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
1,126df183-9908-4df3-80a3-128a10fd6b3a,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
2,19bdf90a-8ca4-4921-8aeb-2bf3423aaf09,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
3,71701d6e-3bba-44c0-af75-271558aedb22,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
4,540317ac-5490-4186-8930-8bfbdaad1055,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
5,97f7f1b1-661f-4076-9352-e698cdf9ef6f,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
6,9995355f-1361-45de-aebb-a7446f8f13f7,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
7,34c775ba-b76d-4ef0-a69f-ab88e90ad151,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
8,1ff86952-9480-45b5-953d-e6056f74659f,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
9,5bd0f37e-7c1f-40ac-b654-97fa66875f70,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None


In [248]:
diagnosticReports.loc[:, diagnosticReports.columns!='data'][:5]

,id,subject.reference,presentedForm.url
0,1ca4c411-116d-49d7-82e0-cde6bff90cfd,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
1,126df183-9908-4df3-80a3-128a10fd6b3a,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
2,19bdf90a-8ca4-4921-8aeb-2bf3423aaf09,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
3,71701d6e-3bba-44c0-af75-271558aedb22,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None
4,540317ac-5490-4186-8930-8bfbdaad1055,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc,None


### fhirpack.extraction.base.getAbsolutePaths

In [249]:
absolutePaths=[
#    'Condition.code.coding.code',
   'Condition.id',
    'Condition.subject.reference',
    'Encounter.id',
    'Encounter.participant.individual.reference',
#     'Procedure.code.coding.display',
#     'Procedure.code.coding.code',
    'Procedure.id',
    'Procedure.status',
    'Procedure.performedDateTime'
]

pack.client

result=pack.getPatients(
    ['Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc']
).getAbsolutePaths(absolutePaths)

<SyncFHIRClient http://test.hapifhir.io/baseR4>

SEARCH[Procedure]> : 100%|██████████| 2/2 [00:00<00:00,  7.70it/s]


In [250]:
result['Procedure']

,Procedure.id,Procedure.performedDateTime,Procedure.status
0,f262bafb-789a-44a1-9453-c21065abc4d1,None,completed
1,e65c5378-5d1a-4978-9590-c6c8846dd6d0,None,completed


In [251]:
paths=[
	'valueQuantity.value',
	'valueQuantity.unit',
	'valueQuantity.system',
	'valueQuantity.code',
]

In [252]:
pack.\
	getPatients(['2164033'])\
		.getObservations()\
			.gatherSimplePaths(paths)

SEARCH[Observation]> : 100%|██████████| 1/1 [00:00<00:00,  4.05it/s]


,valueQuantity.value,valueQuantity.unit,valueQuantity.system,valueQuantity.code
0,82.5,kg,http://unitsofmeasure.org,kg


In [253]:
result["Condition"]

,Condition.id,Condition.subject.reference
0,b8a1a1c4-f973-4b6b-a0ea-3d04bc4c8823,Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc


In [254]:
result["Encounter"]

,Encounter.id,Encounter.participant.individual.reference
0,b9f19065-c8e6-40b9-b368-b09a8fb9a8d6,[Practitioner/00000171-0929-2892-0000-00000001...
1,174d8c79-5517-4f80-967f-3939f943b2c8,[Practitioner/00000171-0929-2892-0000-00000001...
2,3f3a061a-5df2-4dc0-872c-e1e4b88955a1,[Practitioner/00000171-0929-2892-0000-00000001...
3,2b8cb0e7-5b1e-4769-a418-bf9e6dd86611,[Practitioner/00000171-0929-2892-0000-00000001...
4,36fb900c-9ff7-4131-812c-536ab50a0a5c,[Practitioner/00000171-0929-2892-0000-00000000...
5,9911ef15-1b9a-4fbd-b95d-05bd4f060269,[Practitioner/00000171-0929-2892-0000-00000001...
6,f3b1e501-aac6-4480-bf35-38474563069a,[Practitioner/00000171-0929-2892-0000-00000001...
7,969f9166-4f9a-4be6-9257-ed00b295a408,[Practitioner/00000171-0929-2892-0000-00000001...
8,806264f4-4f45-4105-a5bc-8e9a9c49b119,[Practitioner/00000171-0929-2892-0000-00000001...
9,40166ee5-0bc3-4819-96c9-e07dd26e938b,[Practitioner/00000171-0929-2892-0000-00000001...


In [255]:
result["Procedure"]

,Procedure.id,Procedure.performedDateTime,Procedure.status
0,f262bafb-789a-44a1-9453-c21065abc4d1,None,completed
1,e65c5378-5d1a-4978-9590-c6c8846dd6d0,None,completed


In [256]:
result['Encounter'].explode(
	'Encounter.participant.individual.reference'
)\
	.rename(
		columns={"Encounter.participant.individual.reference":"data"}
		)\
			.dropna()\
				.getResources()[:5]\
					.gatherSimplePaths(['name.given','name.family'])

GET[None]> : 100%|██████████| 11/11 [00:03<00:00,  3.47it/s]


,name.given,name.family
0,[[Willy639]],[Rutherford999]
1,[[Willy639]],[Rutherford999]
2,[[Willy639]],[Rutherford999]
3,[[Willy639]],[Rutherford999]
4,[[Lilla884]],[Mills423]


### fhirpack.extraction.getImagingStudies

In [257]:
conds=pack.getConditions(
	searchParams={
		"code":"C61",
		# "_content": "leber"
	}
	)
conds

SEARCH[Condition]> : 100%|██████████| 1/1 [00:00<00:00,  2.73it/s]


,data,Condition.id
0,"{'resourceType': 'Condition', 'id': '1849657',...",1849657


In [258]:
pack\
	.getImagingStudies(searchParams={"endpoint:missing":False})\
		.gatherSimplePaths(['series.description','series.uid','identifier.value'])

# use .getDICOMInstances().sendDICOMToFiles() to download DICOM files

SEARCH[ImagingStudy]> : 100%|██████████| 4/4 [00:00<00:00, 12.94it/s]


,series.description,series.uid,identifier.value
0,None,[IdType-2],[Identifier-117]
1,None,[IdType-5],[Identifier-118]
2,None,None,None
3,None,[1.2.276.0.7230010.3.1.3.8323329.10.1594989066...,[urn:oid:1.2.826.0.1.3680043.8.498.94636677472...


### Transformation

### fhirpack.transformation.base.gatherKeys

In [259]:
patients.gatherKeys().data.explode().value_counts()[:10]

name.given        6
name.family       5
telecom.use       5
resourceType      4
id                4
telecom.system    4
gender            4
name              4
birthDate         4
telecom.value     4
Name: data, dtype: int64

In [260]:
pack.getPatients(
    ['e92abcdc-a300-41e0-aa0f-378daebe25dc']
    )\
        .gatherKeys(['valueString'])\
            .explode()

GET[Patient]> : 100%|██████████| 1/1 [00:00<00:00,  2.82it/s]


,data
0,resourceType
0,id
0,meta
0,meta.versionId
0,meta.lastUpdated
...,...
0,communication.language.coding
0,communication.language.coding.system
0,communication.language.coding.code
0,communication.language.coding.display


### fhirpack.transformation.base.valuesForKeys

In [11]:
pack.getPatients(['e92abcdc-a300-41e0-aa0f-378daebe25dc'])\
        .gatherValuesForKeys(['valueString'])

GET[Patient]> : 100%|██████████| 1/1 [00:00<00:00,  3.80it/s]


,data
0,"[White, Not Hispanic or Latino, Nanci249 Gleic..."


### fhirpack.transformation.base.gatherReferences

In [262]:
pack.gatherReferences(
    ['Patient/9ac622c4-61c8-40b4-b6d9-06cfeb3d6995']
    , recursive=True
)

,referencer,referencee
0,{'reference': 'Patient/9ac622c4-61c8-40b4-b6d9...,[Patient/b558da74-7756-4845-87d5-d1cca8b79a62]
1,{'reference': 'Patient/b558da74-7756-4845-87d5...,[]


### fhirpack.transformation.base.gatherText

In [263]:
pack.getPatients(
    ['Patient/1555106'],
    )\
        .getObservations()[:5]\
                .gatherText()

SEARCH[Observation]> :  15%|█▍        | 100/677 [00:00<00:02, 239.22it/s]


,data
0,[Alkaline phosphatase [Enzymatic activity/volu...
1,"[laboratory, Protein [Mass/volume] in Serum or..."
2,"[laboratory, Glomerular filtration rate/1.73 s..."
3,"[MCH [Entitic mass] by Automated count, labora..."
4,"[laboratory, Carbon dioxide, total [Moles/volu..."


### fhirpack.transformation.base.gatherDates

In [264]:
pack.getPatients(['Patient/1555106'],)\
        .getObservations()[:5]\
                .gatherValuesForKeys(['issued'])

SEARCH[Observation]> :  15%|█▍        | 100/677 [00:00<00:02, 264.53it/s]


,data
0,[2020-03-18T21:53:14.197-04:00]
1,[2020-03-19T21:53:14.197-04:00]
2,[2020-03-19T21:53:14.197-04:00]
3,[2020-03-14T21:53:14.197-04:00]
4,[2020-03-19T21:53:14.197-04:00]


In [265]:
pack.getPatients(['Patient/1555106'],)\
        .getObservations()[:5]\
                .gatherDates()

SEARCH[Observation]> :  15%|█▍        | 100/677 [00:00<00:02, 263.47it/s]


,dates
0,"[2020-03-18T21:53:14-04:00, 2020-03-18T21:53:1..."
1,"[2020-03-19T21:53:14-04:00, 2020-03-19T21:53:1..."
2,"[2020-03-19T21:53:14-04:00, 2020-03-19T21:53:1..."
3,"[2020-03-14T21:53:14-04:00, 2020-03-14T21:53:1..."
4,"[2020-03-19T21:53:14-04:00, 2020-03-19T21:53:1..."


### Load

In [266]:
pack.validate(['Patient/1555106'])\
    .sendResourcesToFiles(['./data.ignore'])

array([ True])

### Utils

### Server Profiling

In [118]:
pack.countServerResources()

,resourceType,count
0,Account,510
1,ActivityDefinition,284
2,AdverseEvent,116
3,AllergyIntolerance,9356
4,Appointment,29636
...,...,...
141,TestReport,11
142,TestScript,33
143,ValueSet,3126
144,VerificationResult,9


### Validation

In [267]:
pack.validate(['Patient/1555106'])

,data
0,"{'resourceType': 'Patient', 'id': '1555106', '..."


### FHIR PACK Internals

#### FHIRPy

### URL Parsing

In [87]:
import fhirpy.base.utils as fpu
url=fpu.parse_pagination_url('/app/FHIR/r4/Condition?code=C61&_count=1&_sort=-onset-date&identifier=%7C&_Pagination=eyJvZmZzZXQiOjIwfQ%3D%3D')
url

('/app/FHIR/r4/Condition',
 {'code': ['C61'],
  '_count': ['1'],
  '_sort': ['-onset-date'],
  'identifier': ['|'],
  '_Pagination': ['eyJvZmZzZXQiOjIwfQ==']})

### Raw fetch_resource()

In [88]:
pack.client._fetch_resource(
	"Condition/1499041"
)

{'resourceType': 'Condition',
 'id': '1499041',
 'meta': {'versionId': '1',
  'lastUpdated': '2020-10-05T16:29:07.960+00:00',
  'source': '#0sjLd8ZeFHHrRHaX'},
 'clinicalStatus': {'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/condition-clinical',
    'code': 'active'}]},
 'verificationStatus': {'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/condition-ver-status',
    'code': 'confirmed'}]},
 'category': [{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/condition-category',
     'code': 'encounter-diagnosis',
     'display': 'Encounter Diagnosis'}]}],
 'code': {'coding': [{'system': 'http://snomed.info/sct',
    'code': '22298006',
    'display': 'Myocardial Infarction'}],
  'text': 'Myocardial Infarction'},
 'subject': {'reference': 'Patient/1465609'},
 'onsetDateTime': '2020-04-01T09:52:49-04:00',
 'recordedDate': '2020-04-01T09:52:49-04:00',
 'asserter': {'reference': 'Practitioner/1498740'}}

### Raw client.execute()

In [89]:
pack.client.execute('Condition',
method='get',
params={
	'code':'C61',
	'_sort':'-onset-date',
	'_count': 1,
	'_total':'accurate'
}
).total

pack.client.execute(
	'DiagnosticReport/_search',
	method="post",
	data={"_id": "9fa9ab4c-9e1c-46c9-9bec-ec01b3215716"},
	params={"_count": 1},
)

1

{'resourceType': 'Bundle',
 'id': 'af8b2454-29bd-4fe3-b350-733131e9383a',
 'meta': {'lastUpdated': '2022-07-13T08:04:01.285+00:00'},
 'type': 'searchset',
 'link': [{'relation': 'self',
   'url': 'https://hapi.fhir.org/baseR4/DiagnosticReport/_search'},
  {'relation': 'next',
   'url': 'https://hapi.fhir.org/baseR4?_getpages=af8b2454-29bd-4fe3-b350-733131e9383a&_getpagesoffset=1&_count=1&_pretty=true&_bundletype=searchset'}],
 'entry': [{'fullUrl': 'https://hapi.fhir.org/baseR4/DiagnosticReport/6630582',
   'resource': {'resourceType': 'DiagnosticReport',
    'id': '6630582',
    'meta': {'versionId': '1',
     'lastUpdated': '2022-07-05T06:03:55.922+00:00',
     'source': '#VSn1gQals3CWesRQ'},
    'text': {'status': 'generated',
     'div': '<div xmlns="http://www.w3.org/1999/xhtml"><div class="hapiHeaderText"> Untitled Diagnostic Report </div><table class="hapiPropertyTable"><tbody><tr><td>Status</td><td>REGISTERED</td></tr><tr><td>Issued</td><td> 15 May 2013 18:32:52 </td></tr><tr><

### Customization

### fhirpack.custom.extraction.base

In [ ]:
# the fhirpack.custom package is guaranteed to never be used by us
# use it to customize fhirpack

# pack.unimplementedPluginBaseExtractorMethod()
# pack.unimplementedPluginBaseTransformerMethod()
# pack.unimplementedPluginBaseLoaderMethod()

# pack.unimplementedPluginSampleExtractorMethod()
# pack.unimplementedPluginSampleTransformerMethod()
# pack.unimplementedPluginSampleLoaderMethod()